# Mapa por regiones RGB

In [1]:
import pandas as pd
#import json
import pycountry
import geopandas as gpd
import numpy as np
import math
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap
from mpl_toolkits.axes_grid1 import make_axes_locatable

def get_color(color_rgb):
	(r,g,b) = color_rgb
	if not math.isnan(r):
		return mpl.colors.rgb2hex(color_rgb)
	else:
		return np.nan

def create_map_RGB(fname, map_fname, img_title=''):	
	#f = json.load(open(fname, "r", encoding="utf8"))
	#regDF = pd.DataFrame.from_dict(f, orient="index", columns=["R", "G", "B"])
    regDF = pd.DataFrame.from_csv(f)
	
	# Iguala el country code de Twitter con el Country Code (iso_a2)del mapa de geopandas (iso_a3)
    countries = {}
	for cc  in regDF.index:
		if cc is None:
			continue
		if cc.lower() == "none":
			continue
		iso_a3 = pycountry.countries.get(alpha_2=cc)
		if  iso_a3 is None:
			continue
			#print(iso_a3)
		countries[cc] = iso_a3.alpha_3

	regDF = regDF.rename(index=countries)
    print(regDF)
	regDF['iso_a3'] = regDF.index

	# definir el mapa con geopandas
	world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))
	world = world[(world.pop_est>0) & (world.name!="Antarctica")]
	world['R'] = np.nan
	world['G'] = np.nan
	world['B'] = np.nan
	world['RGB'] = np.nan
	world['color'] = np.nan

	#ajustar el nombre de los paises con error en los datos de GEOPANDAS
	world.loc[world['name'] == 'France', ['iso_a3']] = 'FRA'
	world.loc[world['name'] == 'Norway', ['iso_a3']] = 'NOR'

	# actualiza los datos del mapa con los conteos por país
	print("Updating CC in Map...")
	for cc in regDF.index:
		#print(df.loc[cc].tweets)
		for cc2 in world.index:
			if world.loc[cc2].iso_a3 == cc:
				world.at[cc2, ['R', 'G', 'B']] = regDF.loc[cc, ['R', 'G', 'B']]
				
	world["RGB"] = world[['R','G','B' ]].apply(tuple, axis=1)
	world['color'] = world["RGB"].apply(get_color)
	world.loc[world['color'].isnull(), "RGB"]=np.nan

	print("Building image...")

	c_df = pd.DataFrame(world[world["color"].notnull()]["RGB"])
	colors_1 = c_df.sort_values(by=["RGB"])["RGB"].to_list()
	cmap_name = 'colors_RGB'
	n_bin=20
	#cmap = LinearSegmentedColormap.from_list(cmap_name, colors_1, N=n_bin)
	cmap= LinearSegmentedColormap.from_list(cmap_name, colors_1)
	plt.register_cmap(cmap=cmap)
	world =  world[(world.name!="Antarctica")]
	fig, ax = plt.subplots(figsize=(15, 10))
	ax.set_axis_off()
	ax.set(title=img_title)
	fig.set_facecolor("w")
	world.plot( column="RGB",  ax=ax,  alpha=0.8, legend=False, edgecolor='k', linewidth=0.2,  cmap="colors_RGB",
			missing_kwds={
				"color": "lightgrey",
				#"edgecolor": "red",
				#"hatch": "///",
				#"label": "Missing values",
			},
	)
	fig.savefig(map_fname, bbox_inches = 'tight', dpi=300)
	print("figure created: ", map_fname )

TabError: inconsistent use of tabs and spaces in indentation (1809284290.py, line 26)

# crear mapa RGB

In [ ]:
img_title = 'Regional Embeddings'
fname = "../figs/colormap-common-voc-semantic-4.csv"
#fname = "region-emo-colors-clustering-umap-12.json"
map_fname = "../figs/fig-" + fname[:-4] + "png"
create_map_RGB(fname, map_fname, img_title)

In [ ]:
# crear mapas RGB
import glob
#img_title = 'Regional Embeddings'
img_title = ''
#for f in  glob.glob("region-emo*.json") + glob.glob("region-lex*.json") : 
for f in  glob.glob("region-lex*.json"): 
	map_fname = "fig-" + f[:-4] + "png"
	print(f, map_fname, img_title)
	create_map_RGB(f, map_fname, img_title)